In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, BaseEnsemble, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import eli5
import shap
from sklearn import model_selection

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [2]:
train = pd.read_csv('../data/CAX_MortgageModeling_Train.csv')
train_len = train.shape[0]

In [3]:
data = pd.read_csv('../data/data_full_categorical_1705.csv')

In [4]:
cnt_cols = []
cat_cols = []
for col in data.columns:
    if data[col].dtype != object and col != 'RESULT' and 'ordinal' not in col and data[col].nunique() > 10:
        cnt_cols.append(col)
    elif 'Z SCORE' not in col:
        cat_cols.append(col)

In [5]:
train = data.iloc[:train_len]
test = data.iloc[train_len:]

In [6]:
cat_cols.remove('Unique_ID')
cat_cols.remove('RESULT')

In [7]:
train2 = pd.concat([train[train.RESULT == 0].sample(frac=.3,random_state=123),train[train.RESULT == 1]],axis=0).sample(frac=1,random_state=123)

In [8]:
kf = KFold(n_splits=5,random_state=123,shuffle=True)
for train_index, test_index in kf.split(train):
    break

In [9]:
from bayes_opt import BayesianOptimization

In [30]:
traincols = list(data.columns)
traincols.remove('RESULT')

In [51]:
def train_catboost(max_depth,iterations,learning_rate):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'iterations': int(iterations)
    }
    clf = CatBoostClassifier(**params, loss_function= 'Logloss', eval_metric='F1',random_seed=42,cat_features=np.arange(len(cat_cols)),verbose=0)
    score = cross_val_score(clf, train2[cat_cols+cnt_cols], train2.RESULT, scoring='f1', cv=KFold(5)).mean()
    print("F1 {:.3f} params {}".format(score, params))
    return score

In [52]:
bounds = {
    'max_depth':(5,10),
    'iterations': (500,5000),
    'learning_rate':(0.01, 0.25)
}

In [53]:
optimizer = BayesianOptimization(
    f=train_catboost,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=20)

|   iter    |  target   | iterat... | learni... | max_depth |
-------------------------------------------------------------


KeyboardInterrupt: 

dtype('int64')